In [1]:
import re
import gc
import string
from tqdm import tqdm
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import seaborn as sns
color = sns.color_palette()
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 

import numpy as np 
import pandas as pd
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
# conda install -c conda-forge wordcloud

# import spacy
# from collections import Counter, defaultdict
# import en_core_web_sm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.pipeline import Pipeline

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dropout, SpatialDropout1D, Embedding, add, concatenate, Concatenate, Input
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, GlobalMaxPool1D
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from numba import jit, cuda

2023-06-08 13:18:26.472765: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# jit("gpu")
train_data = pd.read_csv('train_1.csv')
# train_data = pd.read_csv('train.csv.zip')
# test_data = pd.read_csv('test.csv.zip')

In [4]:
comments = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [5]:
train_data["comment_text"].head(100)

0     Explanation\nWhy the edits made under my usern...
1     D'aww! He matches this background colour I'm s...
2     Hey man, I'm really not trying to edit war. It...
3     "\nMore\nI can't make any real suggestions on ...
4     You, sir, are my hero. Any chance you remember...
5     "\n\nCongratulations from me as well, use the ...
6          COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7     Your vandalism to the Matt Shirvington article...
8     Sorry if the word 'nonsense' was offensive to ...
9     alignment on this subject and which are contra...
10    "\nFair use rationale for Image:Wonju.jpg\n\nT...
11    bbq \n\nbe a man and lets discuss it-maybe ove...
12    Hey... what is it..\n@ | talk .\nWhat is it......
13    Before you start throwing accusations and warn...
14    Oh, and the girl above started her arguments w...
15    "\n\nJuelz Santanas Age\n\nIn 2002, Juelz Sant...
16    Bye! \n\nDon't look, come or think of comming ...
17     REDIRECT Talk:Voydan Pop Georgiev- Cherno

In [6]:
import regex
df = pd.DataFrame()
#Lemmatize Words
import nltk
def get_pos_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN
    
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

REPLACE_URLS = re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+')
REPLACE_HASH = re.compile(r'#(\w+)')
REPLACE_AT = re.compile(r'@(\w+)')
REPLACE_HTML_TAGS = re.compile(r'<[^>]+>')
REPLACE_DIGITS = re.compile(r'\d+')
#REPLACE_PUNCTUATION = re.compile(r'!\"#+$%&\)*,-./:;<=>?@[\\]^_`{|}~\t\(\\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—')     #[,!@\'\"?\.$%_&#*+-:;]")       #[!\"#$%&\'()*+,-\./:;<=>?@[\\]^_`{|}~]")
REPLACE_PUNCTUATION = regex.compile('[\p{C}|\p{M}|\p{P}|\p{S}|\p{Z}]+', regex.UNICODE)
LEAKY_FEATURE = re.compile(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}") 

STOPWORDS = set(stopwords.words('english'))

sentences = [] #for Word2Vec model
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = REPLACE_HTML_TAGS.sub(' ', text)
    
    text = re.sub(r'&amp;', '&', text)
    
    text = REPLACE_URLS.sub('', text)
    #text = REPLACE_HASH.sub('', text)
    text = REPLACE_AT.sub('', text)
    text = REPLACE_DIGITS.sub('', text)
    text = REPLACE_PUNCTUATION.sub(' ' , text).strip()
    text = LEAKY_FEATURE.sub('', text)
    
    text = [APPO[word] if word in APPO else word for word in text.split()]
    
    text = " ".join(lemmatizer.lemmatize(word.strip(), get_pos_tag(pos_tag([word.strip()])[0][1])) for word in text if word not in STOPWORDS)
    
    #sentences.append(text.split())
    return text

In [7]:
train_data["comment"] = train_data["comment_text"].apply(clean_text)

In [11]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/numanaslam/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [9]:
train_data["comment"]

0     explanation edits make username hardcore metal...
1     aww match background colour seemingly stuck th...
2     hey man really try edit war guy constantly rem...
3     make real suggestion improvement wonder sectio...
4                         sir hero chance remember page
5                congratulation well use tool well talk
6                           cocksucker piss around work
7     vandalism matt shirvington article revert plea...
8     sorry word nonsense offensive anyway intend wr...
9                  alignment subject contrary dulithgow
10    fair use rationale image wonju jpg thanks uplo...
11                       bbq man let discus maybe phone
12    hey talk exclusive group wp taliban good destr...
13    start throw accusation warning let review edit...
14    oh girl start argument stuck nose belong belie...
15    juelz santanas age juelz santana year old come...
16              bye look come think comming back tosser
17      redirect talk voydan pop georgiev cherno